In [1]:
from __future__ import annotations
from tqdm import tqdm
from dataclasses import dataclass
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re
import heapq as heap
import time

import sys
sys.setrecursionlimit(10000)

sns.set()

with open('data.txt') as file:
    data = file.read().splitlines()

In [2]:
def get_data():

    walls = {}
    blizzards_set = set()
    blizzards = []

    for row, line in enumerate(data):
        for col, point in enumerate(line):
            match point:
                case "<" | ">" | "^" | "v":
                    blizzards_set.add((row, col))
                    blizzards.append((point, (row, col)))
                case "#":
                    walls[(row, col)] = point

    return walls, blizzards, blizzards_set


def print_map(walls, blizzards, pos):

    max_width = max([w[1] for w in walls.keys()])
    max_height = max([w[0] for w in walls.keys()])

    b_dict = {}

    for dir, point in blizzards:

        if point in b_dict:
            if isinstance(b_dict[point], int):
                b_dict[point] = b_dict[point] + 1
            else:
                b_dict[point] = 2
        else:
            b_dict[point] = dir

    for row in range(max_height + 1):
        for col in range(max_width + 1):

            if (row, col) in walls:
                print(walls[(row, col)], end="")
            elif (row, col) in b_dict:
                print(b_dict[(row, col)], end="")
            elif (row, col) == pos:
                print('😃', end="")
            else:
                print(".", end="")

        print()


def update_blizzards(blizzards, max_height, max_width):

    new_blizzards = []
    new_blizzards_set = set()

    for dir, cords in blizzards:

        match dir:
            case "^":
                new_cord = (cords[0] - 1 if cords[0] > 1 else max_height - 1, cords[1])
            case "v":
                new_cord = (cords[0] + 1 if cords[0] < max_height - 1 else 1, cords[1])
            case ">":
                new_cord = (cords[0], cords[1] + 1 if cords[1] < max_width - 1 else 1)
            case "<":
                new_cord = (cords[0], cords[1] - 1 if cords[1] > 1 else max_width - 1)

        new_blizzards_set.add(new_cord)
        new_blizzards.append((dir, new_cord))

    return new_blizzards, new_blizzards_set


walls, blizzards, blizzards_set = get_data()

max_width = max([w[1] for w in walls.keys()])
max_height = max([w[0] for w in walls.keys()])

start_pos = (0, 1)
end_pos = (max_height, max_width - 1)

best_solution = 438
best_history = []

cache = {}

def get_shortest_path(
    pos, end_pos, steps, blizzards, blizzards_set, walls, max_height, max_width, history=[]
):

    global best_solution
    global best_history
    history = history.copy()
    history.append(pos)

    key = (steps, pos)

    if key in cache:
        return cache[key]

    steps_left = abs(end_pos[0] - pos[0]) + abs(end_pos[1] - pos[1])
    best_solutions = {}

    if steps + steps_left >= best_solution:
        cache[key] = None
        return None

    if steps_left == 0:
        cache[key] = steps
        if steps < best_solution:
            best_solution = steps
            best_history = history

        print('Found solution ', steps)
        return steps

    # Get next pos
    blizzards, blizzards_set = update_blizzards(blizzards, max_height, max_width)

    new_positions = set(
        [
            pos,
            (max(pos[0] - 1, 0), pos[1]),
            (min(pos[0] + 1, max_height), pos[1]),
            (pos[0], pos[1] + 1),
            (pos[0], pos[1] - 1),
        ]
    )

    new_positions = new_positions - blizzards_set - (walls.keys() | set())
    new_positions = [(pos, abs(end_pos[0] - pos[0]) + abs(end_pos[1] - pos[1])) for pos in new_positions]
    new_positions = sorted(new_positions, key=lambda x: x[1])
    alts = []

    if len(new_positions) == 0:
        return None
    else:
        for new_pos in new_positions:
            alts.append(get_shortest_path(new_pos[0], end_pos, steps + 1, blizzards, blizzards_set, walls, max_height, max_width, history))

    alts = [a for a in alts if a != None]
    best_path = min(alts) if len(alts) > 0 else None
    cache[key] = best_path

    return best_path

step_1 = get_shortest_path(start_pos, end_pos, 0, blizzards, blizzards_set, walls, max_height, max_width)
print(step_1)

# Update blizzards
for _ in range(step_1):
    blizzards, blizzards_set = update_blizzards(blizzards, max_height, max_width)

cache = {}
best_solution = 438
step_2 = get_shortest_path(end_pos, start_pos, 0, blizzards, blizzards_set, walls, max_height, max_width)

# Update blizzards
for _ in range(step_2):
    blizzards, blizzards_set = update_blizzards(blizzards, max_height, max_width)

cache = {}
best_solution = 438

step_3 = get_shortest_path(start_pos, end_pos, 0, blizzards, blizzards_set, walls, max_height, max_width)
print(step_3)

Found solution  437
Found solution  405
Found solution  399
Found solution  398
Found solution  379
Found solution  360
Found solution  347
Found solution  344
Found solution  343
343
Found solution  425
Found solution  396
Found solution  387
Found solution  377
Found solution  363
Found solution  320
Found solution  431
Found solution  419
Found solution  392
Found solution  382
Found solution  380
Found solution  374
Found solution  342
Found solution  336
Found solution  335
Found solution  316
Found solution  297
297


In [4]:
print(step_1 + step_2 + step_3)

960
